# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import math
from scipy import stats
import requests

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'E MNSSLGALAA( TD/CRSL EEATBKN)GQO',
 'calculationPrice': 'close',
 'open': 133.65,
 'openTime': 1638415524590,
 'openSource': 'cfilaiof',
 'close': 129.62,
 'closeTime': 1627034797421,
 'closeSource': 'afcioifl',
 'high': 128.46,
 'highTime': 1647929814772,
 'highSource': 'ryiue dte 5ealie1nmpcd ',
 'low': 132.69,
 'lowTime': 1683421726655,
 'lowSource': 'etye 1ni5u e laeicpdrdm',
 'latestPrice': 128.61,
 'latestSource': 'Close',
 'latestTime': 'December 16, 2020',
 'latestUpdate': 1673505757395,
 'latestVolume': 98265795,
 'iexRealtimePrice': 132.032,
 'iexRealtimeSize': 3,
 'iexLastUpdated': 1678780595802,
 'delayedPrice': 131.576,
 'delayedPriceTime': 1681535242416,
 'oddLotDelayedPrice': 133.62,
 'oddLotDelayedPriceTime': 1666646764408,
 'extendedPrice': 129.37,
 'extendedChange': -0.06,
 'extendedChangePercent': -0.00048,
 'extendedPriceTime': 1611648128055,
 'previousClose': 131.85,
 'previousVolume': 162631304,

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
pe_ratio = data['peRatio']
pe_ratio

39.31

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,119.92,55.12,N/A
1,AAL,17.65,-1.26,N/A
2,AAP,161.49,23.46,N/A
3,AAPL,130.82,40.76,N/A
4,ABBV,105.73,23.18,N/A
...,...,...,...,...
500,YUM,108.64,32.56,N/A
501,ZBH,156.02,-210.46,N/A
502,ZBRA,386.29,44.58,N/A
503,ZION,42.17,17.85,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [8]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,33.880,2,N/A
1,NLOK,21.000,4.04,N/A
2,AIV,4.780,4.56,N/A
3,UNM,23.070,5,N/A
4,BIO,599.250,5.11,N/A
5,KIM,15.700,6.9,N/A
6,ALL,108.000,7.61,N/A
7,MET,46.830,7.7,N/A
8,CPB,48.090,8.14,N/A
9,BIIB,258.130,8.34,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [10]:
portfolio_input()

Enter the value of your portfolio:10000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [11]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,33.880,2,5903
1,NLOK,21.000,4.04,9523
2,AIV,4.780,4.56,41841
3,UNM,23.070,5,8669
4,BIO,599.250,5.11,333
5,KIM,15.700,6.9,12738
6,ALL,108.000,7.61,1851
7,MET,46.830,7.7,4270
8,CPB,48.090,8.14,4158
9,BIIB,258.130,8.34,774


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [13]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']
print(pe_ratio)

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']
print(pb_ratio)
#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']
print(ps_ratio)
# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda
print(ev_to_ebitda)
# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit
print(ev_to_gross_profit)

40.3
34.06
8.31
28.54898953047176
20.51444596107402


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [14]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

In [15]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,121.74,N/A,55.53,N/A,7.59,N/A,7.24,N/A,33.543804,N/A,12.827616,N/A,N/A
1,AAL,17.62,N/A,-1.27,N/A,-1.93,N/A,0.4346,N/A,-6.242904,N/A,1.401899,N/A,N/A
2,AAP,165.96,N/A,23.7,N/A,2.9,N/A,1.13,N/A,11.084049,N/A,2.588711,N/A,N/A
3,AAPL,132.45,N/A,40.85,N/A,34.39,N/A,7.97,N/A,29.462533,N/A,21.499114,N/A,N/A
4,ABBV,108.15,N/A,22.94,N/A,12.7,N/A,4.69,N/A,16.163929,N/A,9.336553,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,111.35,N/A,33.13,N/A,-4,N/A,6.1,N/A,23.800815,N/A,10.207815,N/A,N/A
501,ZBH,156.07,N/A,-205.73,N/A,2.6,N/A,4.57,N/A,25.984740,N/A,7.736193,N/A,N/A
502,ZBRA,394.15,N/A,43.83,N/A,10.57,N/A,4.72,N/A,31.641232,N/A,11.241078,N/A,N/A
503,ZION,43.64,N/A,18.66,N/A,0.9887,N/A,2.37,N/A,10.247007,N/A,2.760199,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [16]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,217.140,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,231.450,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,10.19,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,25.750,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,49.350,N/A,14.35,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,53.740,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,28.770,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,136.380,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
194,FTI,9.990,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,1807.000,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [17]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [18]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [19]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0      0.865347
1      0.140594
2      0.453465
3      0.748515
4      0.439604
         ...   
500    0.655446
501    0.019802
502    0.778218
503    0.362376
504    0.829703
Name: PE Percentile, Length: 505, dtype: object
0       0.728713
1      0.0455446
2        0.40297
3       0.964356
4       0.853465
         ...    
500    0.0415842
501     0.365347
502     0.828713
503    0.0980198
504     0.942574
Name: PB Percentile, Length: 505, dtype: object
0       0.809901
1      0.0316832
2       0.175248
3       0.843564
4       0.677228
         ...    
500     0.767327
501     0.630693
502     0.683168
503          0.4
504     0.934653
Name: PS Percentile, Length: 505, dtype: object
0       0.853465
1      0.0277228
2       0.269307
3       0.823762
4        0.50297
         ...    
500     0.734653
501     0.782178
502     0.839604
503     0.227723
504     0.857426
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0       0.786139
1      0.0534653
2       0.130693
3       0.952

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,121.74,N/A,55.53,0.865347,7.5900,0.728713,7.2400,0.809901,33.543804,0.853465,12.827616,0.786139,N/A
1,AAL,17.62,N/A,-1.27,0.140594,-1.9300,0.0455446,0.4346,0.0316832,-6.242904,0.0277228,1.401899,0.0534653,N/A
2,AAP,165.96,N/A,23.70,0.453465,2.9000,0.40297,1.1300,0.175248,11.084049,0.269307,2.588711,0.130693,N/A
3,AAPL,132.45,N/A,40.85,0.748515,34.3900,0.964356,7.9700,0.843564,29.462533,0.823762,21.499114,0.952475,N/A
4,ABBV,108.15,N/A,22.94,0.439604,12.7000,0.853465,4.6900,0.677228,16.163929,0.50297,9.336553,0.641584,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,111.35,N/A,33.13,0.655446,-4.0000,0.0415842,6.1000,0.767327,23.800815,0.734653,10.207815,0.683168,N/A
501,ZBH,156.07,N/A,-205.73,0.019802,2.6000,0.365347,4.5700,0.630693,25.984740,0.782178,7.736193,0.50297,N/A
502,ZBRA,394.15,N/A,43.83,0.778218,10.5700,0.828713,4.7200,0.683168,31.641232,0.839604,11.241078,0.714851,N/A
503,ZION,43.64,N/A,18.66,0.362376,0.9887,0.0980198,2.3700,0.4,10.247007,0.227723,2.760199,0.142574,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [20]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,121.74,N/A,55.53,0.865347,7.5900,0.728713,7.2400,0.809901,33.543804,0.853465,12.827616,0.786139,0.808713
1,AAL,17.62,N/A,-1.27,0.140594,-1.9300,0.0455446,0.4346,0.0316832,-6.242904,0.0277228,1.401899,0.0534653,0.059802
2,AAP,165.96,N/A,23.70,0.453465,2.9000,0.40297,1.1300,0.175248,11.084049,0.269307,2.588711,0.130693,0.286337
3,AAPL,132.45,N/A,40.85,0.748515,34.3900,0.964356,7.9700,0.843564,29.462533,0.823762,21.499114,0.952475,0.866535
4,ABBV,108.15,N/A,22.94,0.439604,12.7000,0.853465,4.6900,0.677228,16.163929,0.50297,9.336553,0.641584,0.62297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,111.35,N/A,33.13,0.655446,-4.0000,0.0415842,6.1000,0.767327,23.800815,0.734653,10.207815,0.683168,0.576436
501,ZBH,156.07,N/A,-205.73,0.019802,2.6000,0.365347,4.5700,0.630693,25.984740,0.782178,7.736193,0.50297,0.460198
502,ZBRA,394.15,N/A,43.83,0.778218,10.5700,0.828713,4.7200,0.683168,31.641232,0.839604,11.241078,0.714851,0.768911
503,ZION,43.64,N/A,18.66,0.362376,0.9887,0.0980198,2.3700,0.4,10.247007,0.227723,2.760199,0.142574,0.246139


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [21]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

In [22]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,23.33,N/A,4.96,0.156436,0.4326,0.049505,0.3852,0.0277228,2.457659,0.0336634,0.369865,0.0039604,0.0542574
1,AIG,38.72,N/A,-6.95,0.106931,0.5435,0.0574257,0.7380,0.0653465,4.445711,0.0415842,0.754244,0.0158416,0.0574257
2,AAL,17.62,N/A,-1.27,0.140594,-1.9300,0.0455446,0.4346,0.0316832,-6.242904,0.0277228,1.401899,0.0534653,0.059802
3,MET,47.59,N/A,7.67,0.164356,0.5845,0.0594059,0.6490,0.0554455,4.299170,0.039604,0.632576,0.0118812,0.0661386
4,HFC,26.49,N/A,-26.40,0.0673267,0.7909,0.0792079,0.3015,0.0138614,6.057583,0.0574257,2.159716,0.112871,0.0661386
5,F,9.13,N/A,-17.27,0.0811881,1.2000,0.133663,0.2749,0.0118812,4.774800,0.0475248,2.141130,0.108911,0.0766337
6,HIG,48.44,N/A,9.78,0.182178,0.9761,0.0930693,0.8315,0.0871287,3.986691,0.0376238,0.783746,0.0178218,0.0835644
7,ALL,109.87,N/A,7.63,0.162376,1.3200,0.165347,0.7581,0.0693069,2.374234,0.0316832,0.695383,0.0138614,0.0885149
8,BA,225.89,N/A,-29.69,0.0653465,-11.1200,0.029703,2.1000,0.356436,-29.978948,0.0138614,-213.436563,0.0019802,0.0934653
9,KSS,39.52,N/A,-59.00,0.0475248,1.3300,0.169307,0.3555,0.0237624,9.614398,0.186139,1.407939,0.0554455,0.0964356


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [23]:
portfolio_input()


Enter the value of your portfolio:1000000


In [24]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

C:\Users\yashw\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,23.33,857,4.96,0.156436,0.4326,0.049505,0.3852,0.0277228,2.457659,0.0336634,0.369865,0.0039604,0.0542574
1,AIG,38.72,516,-6.95,0.106931,0.5435,0.0574257,0.7380,0.0653465,4.445711,0.0415842,0.754244,0.0158416,0.0574257
2,AAL,17.62,1135,-1.27,0.140594,-1.9300,0.0455446,0.4346,0.0316832,-6.242904,0.0277228,1.401899,0.0534653,0.059802
3,MET,47.59,420,7.67,0.164356,0.5845,0.0594059,0.6490,0.0554455,4.299170,0.039604,0.632576,0.0118812,0.0661386
4,HFC,26.49,755,-26.40,0.0673267,0.7909,0.0792079,0.3015,0.0138614,6.057583,0.0574257,2.159716,0.112871,0.0661386
5,F,9.13,2190,-17.27,0.0811881,1.2000,0.133663,0.2749,0.0118812,4.774800,0.0475248,2.141130,0.108911,0.0766337
6,HIG,48.44,412,9.78,0.182178,0.9761,0.0930693,0.8315,0.0871287,3.986691,0.0376238,0.783746,0.0178218,0.0835644
7,ALL,109.87,182,7.63,0.162376,1.3200,0.165347,0.7581,0.0693069,2.374234,0.0316832,0.695383,0.0138614,0.0885149
8,BA,225.89,88,-29.69,0.0653465,-11.1200,0.029703,2.1000,0.356436,-29.978948,0.0138614,-213.436563,0.0019802,0.0934653
9,KSS,39.52,506,-59.00,0.0475248,1.3300,0.169307,0.3555,0.0237624,9.614398,0.186139,1.407939,0.0554455,0.0964356
